# Código para descobrir as notas de matemática do ENEM 2016

In [32]:
# Importing importants modules and setting display options

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', -1)
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_info_columns', -1)

In [43]:
## Importing data
df = pd.read_csv('train.csv', index_col=None)
test = pd.read_csv('test.csv', index_col=None)

testInscricao = test['NU_INSCRICAO'].values

test.drop(['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 
           'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 'Q027'], axis=1, inplace=True)
print(df.columns)
test.sample(10)




# Manipulando os dataset de treino
print(df.shape)
df['NU_NOTA_MT'].fillna(0, inplace=True)
y = df['NU_NOTA_MT'].to_numpy()
df = df[list(test.columns)]
print(df.shape)

df['TP_SEXO'] = df['TP_SEXO'].astype('category')
df = pd.get_dummies(data=df, columns=['TP_SEXO'], drop_first=True)

for column in ['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']:
    df[column] = df[column].astype('category')
    df = pd.get_dummies(data=df, columns=[column], drop_first=True)
    
df.fillna(0, inplace=True)
    
print(df.shape)

for i in range(df.shape[1]):
    pd.to_numeric(df[df.columns[i]], errors='coerce')

df.head()


Index(['Unnamed: 0', 'NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL',
       ...
       'Q041', 'Q042', 'Q043', 'Q044', 'Q045', 'Q046', 'Q047', 'Q048', 'Q049',
       'Q050'],
      dtype='object', length=167)
(13730, 167)
(13730, 38)
(13730, 72)


,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DISLEXIA,IN_DISCALCULIA,IN_SABATISTA,IN_GESTANTE,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,TP_SEXO_M,Q001_B,Q001_C,Q001_D,Q001_E,Q001_F,Q001_G,Q001_H,Q002_B,Q002_C,Q002_D,Q002_E,Q002_F,Q002_G,Q002_H,Q006_B,Q006_C,Q006_D,Q006_E,Q006_F,Q006_G,Q006_H,Q006_I,Q006_J,Q006_K,Q006_L,Q006_M,Q006_N,Q006_O,Q006_P,Q006_Q,Q024_B,Q024_C,Q024_D,Q024_E,Q025_B,Q026_B,Q026_C,Q047_B,Q047_C,Q047_D,Q047_E
0,43,24,1,1,1,4,1,0,0,0,0,0,0,0,0,0,1,1,1,436.3,495.4,581.2,1,1.0,120.0,120.0,120.0,80.0,80.0,520.0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,23,17,3,1,2,0,2,0,0,0,0,0,0,0,0,0,1,1,1,474.5,544.1,599.0,1,1.0,140.0,120.0,120.0,120.0,80.0,580.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,23,21,3,1,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,33,25,0,1,1,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0
4,13,28,2,1,1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [44]:
# Manipulando o dataset de teste

test['TP_SEXO'] = test['TP_SEXO'].astype('category')
test = pd.get_dummies(data=test, columns=['TP_SEXO'], drop_first=True)

for column in ['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']:
    test[column] = test[column].astype('category')
    test = pd.get_dummies(data=test, columns=[column], drop_first=True)
    
    
test.fillna(0, inplace=True)
    
print(test.shape)

for i in range(test.shape[1]):
    pd.to_numeric(test[test.columns[i]], errors='coerce')


y0 = test.values
print(test.shape)
test.head()

(4576, 72)
(4576, 72)


,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DISLEXIA,IN_DISCALCULIA,IN_SABATISTA,IN_GESTANTE,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,TP_SEXO_M,Q001_B,Q001_C,Q001_D,Q001_E,Q001_F,Q001_G,Q001_H,Q002_B,Q002_C,Q002_D,Q002_E,Q002_F,Q002_G,Q002_H,Q006_B,Q006_C,Q006_D,Q006_E,Q006_F,Q006_G,Q006_H,Q006_I,Q006_J,Q006_K,Q006_L,Q006_M,Q006_N,Q006_O,Q006_P,Q006_Q,Q024_B,Q024_C,Q024_D,Q024_E,Q025_B,Q026_B,Q026_C,Q047_B,Q047_C,Q047_D,Q047_E
0,41,22,3,1,1,5,1,0,0,0,0,0,0,0,0,0,1,1,1,464.8,443.5,431.8,0,1.0,120.0,80.0,80.0,100.0,40.0,420.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,21,26,3,1,1,8,1,0,0,0,0,0,0,0,0,0,1,1,1,391.1,491.1,548.0,1,1.0,120.0,120.0,120.0,120.0,100.0,580.0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
2,23,21,1,1,2,0,2,0,0,0,0,0,0,0,0,0,1,1,1,595.9,622.7,613.6,0,1.0,80.0,40.0,40.0,80.0,80.0,320.0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,15,27,3,1,1,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
4,41,18,1,1,2,0,2,0,0,0,0,0,0,0,0,0,1,1,1,592.9,492.6,571.4,1,1.0,100.0,80.0,60.0,80.0,0.0,320.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0


In [45]:
## EDA
print(df.shape)
print(test.shape)
print(df.columns)
test.head()

print(test.columns)



(13730, 72)
(4576, 72)
Index(['CO_UF_RESIDENCIA', 'NU_IDADE', 'TP_COR_RACA', 'TP_NACIONALIDADE',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TREINEIRO',
       'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DISLEXIA',
       'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE', 'IN_IDOSO',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'NU_NOTA_CN',
       'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_LINGUA', 'TP_STATUS_REDACAO',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'TP_SEXO_M', 'Q001_B', 'Q001_C',
       'Q001_D', 'Q001_E', 'Q001_F', 'Q001_G', 'Q001_H', 'Q002_B', 'Q002_C',
       'Q002_D', 'Q002_E', 'Q002_F', 'Q002_G', 'Q002_H', 'Q006_B', 'Q006_C',
       'Q006_D', 'Q006_E', 'Q006_F', 'Q006_G', 'Q006_H', 'Q006_I', 'Q006_J',
       'Q006_K', 'Q006_L', 'Q006_M', 'Q006_N', 'Q006_O', 'Q006_P', 'Q006_Q',
       'Q024_B', 'Q024_C', 'Q024_D', 'Q024_E', 'Q025_B', 'Q026_B', 'Q026_C',
       'Q047_

In [46]:
X = df.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

reg = RandomForestRegressor()

# Hyperparameter tuning

param_grid = {'n_estimators': np.arange(1, 150), 'min_samples_split': np.arange(2, 50), 'min_samples_leaf': np.arange(2, 50)}
grid = GridSearchCV(reg, param_grid, cv=5, error_score='raise')
grid.fit(X, y)
param = grid.best_params_

print(param)

In [47]:
#Fit and predict model


reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

score = mean_squared_error(y_test, y_pred)
print(score)
print(y_pred)
df_pred = pd.DataFrame({'y-pred': y_pred, 'y': y_test})

print(df_pred.head())

C:\Users\guilh\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


4325.997628210731
[  0.   413.27 478.04 ... 398.07 438.2    0.  ]
   y-pred      y
0    0.00    0.0
1  413.27  362.0
2  478.04  493.1
3  478.38  370.8
4  588.85  489.6


In [48]:
nota_mt = reg.predict(y0)



In [51]:
answer = pd.DataFrame({'NU_INSCRICAO': testInscricao, 'NU_NOTA_MT': nota_mt})
print(answer.head())
answer.to_csv('answer.csv', index=False)

                               NU_INSCRICAO  NU_NOTA_MT
0  73ff9fcc02f0a99919906c942c2e1a1042cdcf98      416.88
1  71a95f9f1b91a82c65ad94abbdf9f54e6066f968      454.52
2  b38a03232f43b11c9d0788abaf060f7366053b6d      594.76
3  70b682d9a3636be23f6120fa9d6b164eb3c6002d        0.00
4  715494628a50142ce8cb17191cfe6d0f3cae0934      554.92
